In [115]:
%pip install pandas matplotlib seaborn scikit-learn -q
%pip install imbalanced-learn -q


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [116]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder, StandardScaler
from imblearn.over_sampling import SMOTE
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import SimpleImputer


df = pd.read_csv('mcdonalds.csv')


In [117]:
df.head()

,_id,active,name,code,country,timezone,address,city,neighborhood,cep,...,services.mcDelivery,services.timeExtended,services.mcParty,services.playPlace,services.parking,services.wifi,services.wheelchairAccess,services.dessertCenter,services.shoppingCenter,generalHour.daysOfWeek
0,61d7fcbda283bebac83e89c9,True,SOBRADINHO - SBR,SBR,BR,America/Sao_Paulo,"QD 06 - Área Reservada 3, s/n",Brasília,Sobradinho,73070035,...,False,True,False,False,True,True,True,False,False,"[{'day': 'mon', 'timePeriods': [{'start': '08:..."
1,62ed34bb7877dc2d6381ea1c,True,Drive Saída Norte - DSN,DSN,BR,America/Sao_Paulo,SPMN EPIA DF 03 Lote 02 Loja 01 Brasília - DF,Brasília,Lago Norte,71560100,...,True,True,False,False,False,False,True,False,False,"[{'day': 'mon', 'timePeriods': [{'start': '10:..."
2,62ed31417877dcd78481e3a7,True,Shopping Iguatemi - BSI,BSI,BR,America/Sao_Paulo,"SHIN CA 4 Lote A, lojas 177/178 – Shopping Igu...",Brasília,Lago Norte,71503504,...,False,False,False,False,False,False,True,True,True,"[{'day': 'mon', 'timePeriods': [{'start': '11:..."
3,62ed30849b780128a34229f8,True,Shopping Boulevard - BSB,BSB,BR,America/Sao_Paulo,"STN Terminal Norte, S/N, Conjunto J, Loja T51 ...",Brasília,Asa Norte,70770100,...,True,False,False,False,False,False,True,False,True,"[{'day': 'mon', 'timePeriods': [{'start': '10:..."
4,62ed2de97877dc4c1281d5ed,True,Asa Norte Eixinho - ASN,ASN,BR,America/Sao_Paulo,EQN 208/209 Bloco A – Lj. 50 – Asa Norte Bras...,Brasília,Asa Norte,70853400,...,True,True,False,False,False,False,True,False,False,"[{'day': 'mon', 'timePeriods': [{'start': '10:..."


In [118]:
df.describe()

,distance,coordinates.longitude,coordinates.latitude,coordinates.lng,coordinates.lat
count,2343.000000,2343.000000,2343.000000,2343.000000,2343.000000
mean,1380.530839,-42.518490,-23.901153,-42.518490,-23.901153
std,1147.258006,9.610560,9.958499,9.610560,9.958499
min,34.005271,-63.876013,-56.114344,-63.876013,-56.114344
25%,898.639183,-47.068181,-23.694802,-47.068181,-23.694802
50%,920.855204,-46.546769,-23.480102,-46.546769,-23.480102
75%,1346.766388,-42.568615,-21.225657,-42.568615,-21.225657
max,11125.013013,46.701319,23.704880,46.701319,23.704880


In [119]:
df.shape

(2343, 39)

In [120]:
rename_dict = {
    'id': 'Identificador',
    'active': 'Ativo',
    'name': 'Nome_do_lugar',
    'code': 'Codigo',
    'country': 'Pais',
    'timezone': 'Fuso_horario',
    'address': 'Endereco',
    'city': 'Cidade',
    'neighborhood': 'Bairro',
    'services.mcDelivery': 'Delivery',
    'services.timeExtended': 'Servicos_com_tempo_extimado',
    'services.mcParty': 'Mcfestas_de_servicos',
    'services.playPlace': 'Lugar_de_brincar',
    'services.parking': 'Serviços_de_estacionamento',
    'services.wifi': 'servicos_com_Wifi',
    'services.wheelchairAccess': 'Acessibilidade',
    'services.dessertCenter': 'Centro_de_sobremesas',
    'services.shoppingCenter': 'Shopping_center',
    'generalHour.daysOfWeek': 'horas_dias_da_semana',
}
df = df.rename(columns=rename_dict)


In [121]:
df.head()

,_id,Ativo,Nome_do_lugar,Codigo,Pais,Fuso_horario,Endereco,Cidade,Bairro,cep,...,Delivery,Servicos_com_tempo_extimado,Mcfestas_de_servicos,Lugar_de_brincar,Serviços_de_estacionamento,servicos_com_Wifi,Acessibilidade,Centro_de_sobremesas,Shopping_center,horas_dias_da_semana
0,61d7fcbda283bebac83e89c9,True,SOBRADINHO - SBR,SBR,BR,America/Sao_Paulo,"QD 06 - Área Reservada 3, s/n",Brasília,Sobradinho,73070035,...,False,True,False,False,True,True,True,False,False,"[{'day': 'mon', 'timePeriods': [{'start': '08:..."
1,62ed34bb7877dc2d6381ea1c,True,Drive Saída Norte - DSN,DSN,BR,America/Sao_Paulo,SPMN EPIA DF 03 Lote 02 Loja 01 Brasília - DF,Brasília,Lago Norte,71560100,...,True,True,False,False,False,False,True,False,False,"[{'day': 'mon', 'timePeriods': [{'start': '10:..."
2,62ed31417877dcd78481e3a7,True,Shopping Iguatemi - BSI,BSI,BR,America/Sao_Paulo,"SHIN CA 4 Lote A, lojas 177/178 – Shopping Igu...",Brasília,Lago Norte,71503504,...,False,False,False,False,False,False,True,True,True,"[{'day': 'mon', 'timePeriods': [{'start': '11:..."
3,62ed30849b780128a34229f8,True,Shopping Boulevard - BSB,BSB,BR,America/Sao_Paulo,"STN Terminal Norte, S/N, Conjunto J, Loja T51 ...",Brasília,Asa Norte,70770100,...,True,False,False,False,False,False,True,False,True,"[{'day': 'mon', 'timePeriods': [{'start': '10:..."
4,62ed2de97877dc4c1281d5ed,True,Asa Norte Eixinho - ASN,ASN,BR,America/Sao_Paulo,EQN 208/209 Bloco A – Lj. 50 – Asa Norte Bras...,Brasília,Asa Norte,70853400,...,True,True,False,False,False,False,True,False,False,"[{'day': 'mon', 'timePeriods': [{'start': '10:..."


In [122]:
df.describe()

,distance,coordinates.longitude,coordinates.latitude,coordinates.lng,coordinates.lat
count,2343.000000,2343.000000,2343.000000,2343.000000,2343.000000
mean,1380.530839,-42.518490,-23.901153,-42.518490,-23.901153
std,1147.258006,9.610560,9.958499,9.610560,9.958499
min,34.005271,-63.876013,-56.114344,-63.876013,-56.114344
25%,898.639183,-47.068181,-23.694802,-47.068181,-23.694802
50%,920.855204,-46.546769,-23.480102,-46.546769,-23.480102
75%,1346.766388,-42.568615,-21.225657,-42.568615,-21.225657
max,11125.013013,46.701319,23.704880,46.701319,23.704880


In [123]:
df['servicos_com_Wifi'] = df['servicos_com_Wifi'].fillna(0)
df['Acessibilidade'] = df['Acessibilidade'].fillna(0)


In [124]:
features_corrigido = ['servicos_com_Wifi', 'Acessibilidade', 'Delivery', 'Servicos_com_tempo_extimado']
X = df[features_corrigido]
y = df['Delivery'].map({True: 1, False: 0})


In [125]:
# Split dos dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Impute missing values in X_train before SMOTE
imputer = SimpleImputer(strategy='mean')  # ou 'median', 'most_frequent', etc.
X_train_imputed = pd.DataFrame(imputer.fit_transform(X_train), columns=X_train.columns)

# Aplicação do SMOTE
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train_imputed, y_train)


In [126]:
scaler = StandardScaler()
X_resampled_scaled = scaler.fit_transform(X_resampled)
X_test_scaled = scaler.transform(X_test)


In [127]:
with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)


In [128]:
param_grid = {
    'n_estimators': [200, 250, 300],
    'max_depth': [10, 12, 15],
    'min_samples_split': [5, 10],
    'min_samples_leaf': [2, 5],
    'class_weight': ['balanced', None]
}
grid_search = GridSearchCV(RandomForestClassifier(random_state=42), param_grid, cv=5, scoring='f1_macro')
grid_search.fit(X_resampled_scaled, y_resampled)
rf = grid_search.best_estimator_


In [129]:
y_pred = rf.predict(X_test_scaled)
print("Matriz de confusão:")
print(confusion_matrix(y_test, y_pred))
print("Relatório de classificação:")
print(classification_report(y_test, y_pred))
print("Acurácia:", accuracy_score(y_test, y_pred))


Matriz de confusão:
[[382   0]
 [  0  87]]
Relatório de classificação:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       382
           1       1.00      1.00      1.00        87

    accuracy                           1.00       469
   macro avg       1.00      1.00      1.00       469
weighted avg       1.00      1.00      1.00       469

Acurácia: 1.0


In [130]:
with open('treinamentomc.pkl', 'wb') as file:
    pickle.dump(rf, file)
